### Basic Example

In [ ]:
# import marshal
import requests # Send post to trigger function in OSCAR.
import json     # To generate the post payload.
import inspect  # To get source code as string.

In [ ]:
# Code to be executed.
def func_payload():
    import ROOT
    import os       
    from datetime import datetime

    df = ROOT.RDataFrame(1000)
 
    # Set the random seed and define two columns of the dataset with random numbers.
    ROOT.gRandom.SetSeed(1)
    df_1 = df.Define("gaus", "gRandom->Gaus(10, 1)").Define("exponential", "gRandom->Exp(10)")
 
    # Book an histogram for each column
    h_gaus = df_1.Histo1D(("gaus", "Normal distribution", 50, 0, 30), "gaus")
    h_exp = df_1.Histo1D(("exponential", "Exponential distribution", 50, 0, 30), "exponential")
 
    # Plot the histograms side by side on a canvas
    c = ROOT.TCanvas("TestCanvasOscasr", "TestCanvasOscasr", 800, 400)
    c.Divide(2, 1)
    c.cd(1)
    h_gaus.DrawCopy()
    c.cd(2)
    h_exp.DrawCopy()
 
    # Save the canvas
    #print(os.getenv('TMP_OUTPUT_DIR'))
    c.SaveAs(str(os.getenv('TMP_OUTPUT_DIR')) + "/payloadtest__" + str(datetime.now().time())  + "__.png")

In [ ]:
#json_payload = json.dumps({
#    'func_payload': marshal.dumps(func_payload.__code__)
#})

text_function = inspect.getsource(func_payload)
                                 
print(text_function)

# Generate function from text
#exec(text_function) 
#func_payload() # It works.

In [ ]:
# Create payload and send to oscar through a POST request
import base64
from requests.auth import HTTPBasicAuth
dict_payload = {"func": str(base64.b64encode(bytes(text_function, 'utf-8'))) }


# Get endpoint status
status_response = requests.get('https://localhost/health', 
                               headers={'Accept':'application/json'}, 
                               verify=False)
print('Server Status: ' + status_response.text)


# Get servie token from request. Requires password :/
token_response = requests.get('https://localhost/system/services/root-map', 
                               headers={'Accept':'application/json'}, 
                               auth=HTTPBasicAuth('oscar', 'oscarPassword'),
                               verify=False)
service_token = json.loads(token_response.text)['token']
print(service_token)


# Set endpoint
cluster_endpoint = 'https://localhost/job/root-map' # /job/ is asynchronous.
headers = {
    "Accept": "application/json",
    "Authorization": "Bearer " + service_token
}


# Run job.
requests.post(cluster_endpoint, 
              headers=headers, 
              json=json.dumps(dict_payload), 
              verify=False)

### Get compute graph of data to process

In [ ]:
import ROOT
import os       
from datetime import datetime
df = ROOT.RDataFrame(1000)

# Set the random seed and define two columns of the dataset with random numbers.
ROOT.gRandom.SetSeed(1)
df_1 = df.Define("gaus", "gRandom->Gaus(10, 1)").Define("exponential", "gRandom->Exp(10)")

# Book an histogram for each column
h_gaus = df_1.Histo1D(("gaus", "Normal distribution", 50, 0, 30), "gaus")
h_exp = df_1.Histo1D(("exponential", "Exponential distribution", 50, 0, 30), "exponential")

# Plot the histograms side by side on a canvas
c = ROOT.TCanvas("TestCanvasOscasr", "TestCanvasOscasr", 800, 400)
c.Divide(2, 1)
c.cd(1)
h_gaus.DrawCopy()
c.cd(2)
h_exp.DrawCopy()

In [ ]:
print(ROOT.RDF.SaveGraph(df))

In [ ]:
print("Slots: " + str(df.GetNSlots()))
print("Runs: " + str(df.GetNRuns()))

### ROOT Implementation

In [ ]:
import ROOT
RDataFrame = ROOT.RDF.Experimental.Distributed.OSCAR.RDataFrame
RunGraphs = ROOT.RDF.Experimental.Distributed.RunGraphs

oscarclient = {
    "endpoint": "https://localhost/job/root-map",
    "token": service_token
}

df = RDataFrame(1000, 
                oscarclient=oscarclient)

In [ ]:
# Set the random seed and define two columns of the dataset with random numbers.
ROOT.gRandom.SetSeed(1)
df_1 = df.Define("gaus", "gRandom->Gaus(10, 1)").Define("exponential", "gRandom->Exp(10)")

# Book an histogram for each column
h_gaus = df_1.Histo1D(("gaus", "Normal distribution", 50, 0, 30), "gaus")
h_exp = df_1.Histo1D(("exponential", "Exponential distribution", 50, 0, 30), "exponential")

# Plot the histograms side by side on a canvas
c = ROOT.TCanvas("distrdf002", "distrdf002", 800, 400)
c.Divide(2, 1)
c.cd(1)
h_gaus.DrawCopy()
c.cd(2)
h_exp.DrawCopy()

# Save the canvas
c.SaveAs("distrdf002_OSCAR_connection.png")
print("Saved figure to distrdf002_OSCAR_connection.png")